Predicting survival chances without making pipelines

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
"""
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScalar
from sklearn.preprocessing import StandardScalar
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier


"""

In [4]:
train= pd.read_csv("train.csv")

test= pd.read_csv("test.csv")

train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Parch basically means number of parents/children aboard the Titanic

Whereas sibsp are the number of siblings/spouses aboard the titanic


Emabark is port of embarkation

Siblings include only brother, sister, stepbrother, stepsister

Spouses include husband and wife only

Parch includes parents (mother and father) and child(daughter,son,stepdaughter,stepson)

Some children travelled only with a nanny, hench parch=0 for them



In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [6]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [8]:
train.isnull().sum()

,0
PassengerId,0
Survived,0
Pclass,0
Name,0
Sex,0
Age,177
SibSp,0
Parch,0
Ticket,0
Fare,0


In [14]:
train["PassengerId"].drop_duplicates()

,PassengerId
0,1
1,2
2,3
3,4
4,5
...,...
886,887
887,888
888,889
889,890


In [ ]:
#P class is numerical data with some heirarchy. It shows class of ticket like first class, second class etc



In [16]:
train.drop(columns=["PassengerId","Name","Ticket","Cabin"],inplace=True)

In [17]:
train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [27]:
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test= train_test_split(train.drop(columns=["Survived"],axis=1),train["Survived"],test_size=0.2,random_state=42)

X_train.head()

#X_train.shape

Y_test.shape #1D array


(179,)

In [34]:
#Imputing missing values

from sklearn.impute import SimpleImputer

si= SimpleImputer() # uses all default parameters of simple imputer. What does this mean?

"""

When you instantiate a SimpleImputer object without specifying any parameters, like this:


si = SimpleImputer()

The SimpleImputer will use its default settings for imputing missing values. Here’s how it behaves by default:

Default Behavior of SimpleImputer

1. missing_values:

Default: np.nan
It will look for missing values represented as np.nan (Not a Number) in your data.


2. strategy:

Default: "mean"
It will replace missing values with the mean of the column in which they are located (for numerical data).


3. fill_value:

Default: None
This is only used when strategy='constant'. Since the default strategy is "mean", fill_value is ignored unless you change the strategy.

How It Works Without Explicit Parameters
If you don’t specify any arguments, SimpleImputer will replace all np.nan values in the numerical columns of your dataset with the mean of each respective column.

For example:

import numpy as np
from sklearn.impute import SimpleImputer

# Example data
data = [[1, 2, np.nan],
        [4, np.nan, 6],
        [7, 8, 9]]

# Instantiate and apply the SimpleImputer
si = SimpleImputer()
imputed_data = si.fit_transform(data)

print(imputed_data)
Output:

[[1.  2.  7.5]
 [4.  5.  6. ]
 [7.  8.  9. ]]

Explanation:


Missing values in the third column (np.nan) were replaced with the mean of the column: (6 + 9) / 2 = 7.5.
Missing value in the second column was replaced with its mean: (2 + 8) / 2 = 5.

When to Specify Parameters


You might want to explicitly set parameters for the SimpleImputer if:

1. You need a strategy other than "mean" (e.g., "median", "most_frequent", or "constant").
2. You want to handle missing values represented in a custom way (e.g., 0 or -1 instead of np.nan).

Example:

si = SimpleImputer(strategy="median", missing_values=-1)
This will replace all -1 values with the median of the respective column.

Key Takeaway
If you don’t specify any parameters, SimpleImputer will look for np.nan and replace them with the column-wise mean. This is often a reasonable default for many datasets with numerical features.

"""

si_embarked= SimpleImputer(strategy= "most_frequent") # most_frequent means fill the missing values with the mode, i.e the most occuring value

X_train_age= si.fit_transform(X_train[["Age"]]) #Always pass columns in the form of dataframe with double brackets [[]] in ML Models and fit_transform

X_train_embarked= si_embarked.fit_transform(X_train[["Embarked"]])

X_test_age= si.transform(X_train[["Age"]])

X_test_embarked= si_embarked.transform(X_train[["Embarked"]])


X_train_age







array([[45.5       ],
       [23.        ],
       [32.        ],
       [26.        ],
       [ 6.        ],
       [24.        ],
       [45.        ],
       [29.        ],
       [29.49884615],
       [29.49884615],
       [42.        ],
       [36.        ],
       [33.        ],
       [17.        ],
       [29.        ],
       [50.        ],
       [35.        ],
       [38.        ],
       [34.        ],
       [17.        ],
       [11.        ],
       [61.        ],
       [30.        ],
       [ 7.        ],
       [63.        ],
       [20.        ],
       [29.49884615],
       [29.        ],
       [36.        ],
       [29.49884615],
       [50.        ],
       [27.        ],
       [30.        ],
       [33.        ],
       [29.49884615],
       [29.49884615],
       [ 2.        ],
       [25.        ],
       [51.        ],
       [25.        ],
       [29.49884615],
       [29.49884615],
       [24.        ],
       [18.        ],
       [29.49884615],
       [25

In [32]:
X_train["Age"] #Pandas Series

X_train[["Age"]].shape #Pandas Dataframe


(712, 1)

In [31]:
X_train["Age"].shape



(712,)

In [ ]:
# One Hot Encoding

from sklearn.preprocessing import OneHotEncoding

oe_gender= OneHotEncoding(sparse=False,drop="first",handle_unknown="ignore")

oe_embarked= OneHotEncoding(sparse=False,drop="first",handle_unknown="ignore")



